In [1]:
%load_ext autoreload
%autoreload 2

Dependencies

In [2]:
import pathlib as pb

# Environment
ROOT_PATH = pb.Path('..')
DATA_DIR_PATH = ROOT_PATH / 'data'
CACHE_DIR_PATH = ROOT_PATH / '.cache'
TRANSFORMERS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'transformers'
DATASETS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'datasets'
TEST_DATA_FILE = DATA_DIR_PATH / 'test_data.csv'
TRAIN_DATA_FILE = DATA_DIR_PATH / 'train_data.csv'

# Model Repositories
BERT_RO_MODEL_REPO = 'dumitrescustefan/bert-base-romanian-cased-v1'

In [3]:

import os

os.environ['TRANSFORMERS_CACHE'] = str(TRANSFORMERS_CACHE_DIR_PATH)
os.environ['HF_DATASETS_CACHE'] = str(DATASETS_CACHE_DIR_PATH)
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import torchdata
import torchtext
from torch import Tensor
from torch import nn
from torch import optim
from torch.utils.data import Subset, DataLoader, Dataset
from torch import backends
import typing
import pathlib as pb
import os
import gc
from typing import List, Tuple, Dict, Set
import random
import numpy as np
import pandas as pd
import platform
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score

/home/invokariman/Projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-processing-2-mFR9zZTH-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from utils import get_available_device, read_data
from preprocess import BertPreprocessor
from data import SexismDataset
from models import BertFlatClassModel

Environment Setup

In [6]:
# Use available GPU
DEVICE: torch.device = get_available_device()

# Deterministic experiments
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
np.random.RandomState(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

Dataset & Preprocessing

In [7]:
train_data_raw, test_data_raw = read_data(DATA_DIR_PATH)

In [8]:
train_dataset = SexismDataset(train_data_raw[:1000], BertPreprocessor(train_data_raw, BERT_RO_MODEL_REPO))
test_dataset = SexismDataset(test_data_raw[:1000], BertPreprocessor(test_data_raw, BERT_RO_MODEL_REPO))

Model Training

In [12]:
# Training specifications
batch_size = 16
num_epochs = 10
num_workers = 1
learning_rate = 1e-6

In [13]:
# Instantiate language model
# Inspired from: https://luv-bansal.medium.com/fine-tuning-bert-for-text-classification-in-pytorch-503d97342db2
model = BertFlatClassModel(BERT_RO_MODEL_REPO).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # TODO: weighted loss

In [ ]:
# Inspired from: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

for i, (train_idx, valid_idx) in enumerate(kf.split(train_dataset)):
    # Split the data
    train_subset = Subset(train_dataset, train_idx)
    valid_subset = Subset(train_dataset, valid_idx)

    # Create the dataloaders
    train_loader = DataLoader(
        train_subset,
        batch_size,
    )
    valid_loader = DataLoader(
        train_subset,
        batch_size,
    )

    # ---  Training  ---
    for epoch in range(num_epochs):
        epoch_loss = []
        epoch_accy = []

        for batch_i, batch in enumerate(train_loader):
            # Send batch to GPU
            batch: Dict[str, Tensor] = { k: v.to(DEVICE) for k, v in batch.items()}

            # Make predictions
            y_true: Tensor | np.ndarray = batch['label']
            y_pred: Tensor | np.ndarray = model.forward(batch)

            # Compute the loss
            optimizer.zero_grad()
            loss: Tensor = loss_fn(y_pred, y_true)
            loss.backward()
            optimizer.step()

            # Compute the accuracy
            y_true = y_true.detach().cpu().numpy()
            y_pred = y_pred.detach().argmax(dim=1).cpu().numpy()
            epoch_accy.append(balanced_accuracy_score(y_true, y_pred))

            # Track progress
            epoch_loss.append(loss.detach().cpu().numpy())

        mean_loss = np.array(epoch_loss).mean()
        mean_accy = np.array(epoch_accy).mean()
        print('Epoch {} - Loss: {}, Accuracy: {}'.format(epoch, mean_loss, mean_accy))

    # --- Validation ---

/home/invokariman/Projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-processing-2-mFR9zZTH-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/invokariman/Projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-processing-2-mFR9zZTH-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/invokariman/Projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-processing-2-mFR9zZTH-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/invokariman/Projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-pro

Epoch 0 - Loss: 1.2858831882476807, Accuracy: 0.23703584072931896


/home/invokariman/Projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-processing-2-mFR9zZTH-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 1 - Loss: 1.0489047765731812, Accuracy: 0.2717908584169454
Epoch 2 - Loss: 0.9518353939056396, Accuracy: 0.2733333333333333
Epoch 3 - Loss: 0.9128478765487671, Accuracy: 0.2733333333333333
Epoch 4 - Loss: 0.8942375779151917, Accuracy: 0.2733333333333333
Epoch 5 - Loss: 0.883380651473999, Accuracy: 0.2733333333333333
Epoch 6 - Loss: 0.8758304715156555, Accuracy: 0.2733333333333333
Epoch 7 - Loss: 0.8698782324790955, Accuracy: 0.2733333333333333
Epoch 8 - Loss: 0.8647992014884949, Accuracy: 0.2733333333333333
Epoch 9 - Loss: 0.8602515459060669, Accuracy: 0.2733333333333333
Epoch 0 - Loss: 0.8275637030601501, Accuracy: 0.2986666666666667
Epoch 1 - Loss: 0.8228095769882202, Accuracy: 0.2986666666666667
Epoch 2 - Loss: 0.8186614513397217, Accuracy: 0.2986666666666667
Epoch 3 - Loss: 0.8147711157798767, Accuracy: 0.2986666666666667
Epoch 4 - Loss: 0.8110924363136292, Accuracy: 0.2986666666666667
Epoch 5 - Loss: 0.8075826168060303, Accuracy: 0.2986666666666667
Epoch 6 - Loss: 0.80420571

KeyboardInterrupt: 

In [ ]:
balanced_accuracy_score(y_true.detach().cpu().numpy(), y_pred.argmax(dim=1).detach().cpu().numpy())

0.25

In [ ]:
y_true

tensor([3, 0, 0, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 0, 4, 3, 3, 1, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 0, 1, 3], device='cuda:0')

In [ ]:
print(y_pred.shape, y_pred.argmax(dim=1))

torch.Size([32, 5]) tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0')


In [ ]:
# Free memory
del model
gc.collect()
torch.cuda.empty_cache()